In [1]:
# Importación de las bibliotecas 

import pandas as pd
import time
import sqlite3
import redis

# Parte 1: Volver a cargar los datasets en pandas
# Fuente 1: Kaggle - Video Game Sales: https://www.kaggle.com/datasets/gregorut/videogamesales
# Fuente 2: Kaggle - IMDb Movies extensive dataset: https://www.kaggle.com/datasets/rajugc/imdb-movies-dataset-based-on-genre
# Fuente 3: Yelp Open Dataset: https://www.kaggle.com/datasets/yelp-dataset/yelp-dataset/data

# Carga de Datos de Videojuegos y Cine
try:
    # Dataset de ventas de videojuegos
    df_vgsales = pd.read_csv('vgsales.csv')
    print("Dataset de ventas de videojuegos cargado exitosamente.")
    print(df_vgsales.head())

    # Dataset de de películas
    df_movies = pd.read_csv('IMDb_movies.csv', encoding='latin1')
    print("\nDataset de películas de IMDb cargado exitosamente.")
    print(df_movies.head())

except FileNotFoundError as e:
    print(f"Error: No se encontró el archivo. Asegúrate de que el nombre del archivo es correcto y que lo has subido a Colab.")
    print(e)


# Carga de Datos de Restaurantes (Yelp)
try:
    # Usamos lines=True para leer el archivo correctamente.
    df_yelp_reviews = pd.read_json('yelp_academic_dataset_review.json', lines=True, nrows=500000)
    print("\nDataset de reseñas de Yelp cargado exitosamente.")
    # Nos interesan principalmente el texto de la reseña y las estrellas
    df_yelp_reviews_sample = df_yelp_reviews[['text', 'stars']]
    print(df_yelp_reviews_sample.head())

except FileNotFoundError as e:
    print(f"Error: No se encontró el archivo de Yelp. Asegúrate de que el nombre del archivo es correcto.")
    print(e)

Dataset de ventas de videojuegos cargado exitosamente.
   Rank                      Name Platform    Year         Genre Publisher  \
0     1                Wii Sports      Wii  2006.0        Sports  Nintendo   
1     2         Super Mario Bros.      NES  1985.0      Platform  Nintendo   
2     3            Mario Kart Wii      Wii  2008.0        Racing  Nintendo   
3     4         Wii Sports Resort      Wii  2009.0        Sports  Nintendo   
4     5  Pokemon Red/Pokemon Blue       GB  1996.0  Role-Playing  Nintendo   

   NA_Sales  EU_Sales  JP_Sales  Other_Sales  Global_Sales  
0     41.49     29.02      3.77         8.46         82.74  
1     29.08      3.58      6.81         0.77         40.24  
2     15.85     12.88      3.79         3.31         35.82  
3     15.75     11.01      3.28         2.96         33.00  
4     11.27      8.89     10.22         1.00         31.37  

Dataset de películas de IMDb cargado exitosamente.
    movie_id                         movie_name  year cert

In [22]:
df_vgsales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16598 non-null  int64  
 1   Name          16598 non-null  object 
 2   Platform      16598 non-null  object 
 3   Year          16327 non-null  float64
 4   Genre         16598 non-null  object 
 5   Publisher     16540 non-null  object 
 6   NA_Sales      16598 non-null  float64
 7   EU_Sales      16598 non-null  float64
 8   JP_Sales      16598 non-null  float64
 9   Other_Sales   16598 non-null  float64
 10  Global_Sales  16598 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


In [14]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368300 entries, 0 to 368299
Data columns (total 14 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   movie_id     368300 non-null  object 
 1   movie_name   368296 non-null  object 
 2   year         315052 non-null  object 
 3   certificate  104191 non-null  object 
 4   runtime      259146 non-null  object 
 5   genre        368300 non-null  object 
 6   rating       230938 non-null  float64
 7   description  368300 non-null  object 
 8   director     340931 non-null  object 
 9   director_id  340931 non-null  object 
 10  star         309605 non-null  object 
 11  star_id      316442 non-null  object 
 12  votes        230942 non-null  float64
 13  gross(in $)  25039 non-null   float64
dtypes: float64(3), object(11)
memory usage: 39.3+ MB


In [15]:
df_yelp_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   review_id    500000 non-null  object        
 1   user_id      500000 non-null  object        
 2   business_id  500000 non-null  object        
 3   stars        500000 non-null  int64         
 4   useful       500000 non-null  int64         
 5   funny        500000 non-null  int64         
 6   cool         500000 non-null  int64         
 7   text         500000 non-null  object        
 8   date         500000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 34.3+ MB


In [17]:
df_metacritic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2376 entries, 0 to 2375
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         2376 non-null   object
 1   metascore     2376 non-null   object
 2   release_date  2376 non-null   object
 3   summary       2376 non-null   object
 4   url           2376 non-null   object
dtypes: object(5)
memory usage: 92.9+ KB


In [25]:
# Parte 2: Limpieza de df_vgsales

# 2.1. Limpieza de df_vgsales
print("--- Limpieza de df_vgsales ---")
# Se hace una copia para no modificar el original
df_vgsales_clean = df_vgsales.copy()

# 1. Manejo de valores nulos
# Rellenamos los Publishers faltantes con 'Unknown' (LÍNEA CORREGIDA)
df_vgsales_clean['Publisher'] = df_vgsales_clean['Publisher'].fillna('Unknown')

# Eliminamos las filas donde el año es nulo, ya que es un dato importante
df_vgsales_clean.dropna(subset=['Year'], inplace=True)

# 2. Corrección de tipos de datos
# Convertimos el año de float (ej. 2006.0) a entero (ej. 2006)
df_vgsales_clean['Year'] = df_vgsales_clean['Year'].astype(int)

# 3. Verificación
print("Información del DataFrame df_vgsales después de la limpieza:")
df_vgsales_clean.info()

print("\nPrimeras filas del DataFrame limpio:")
print(df_vgsales_clean.head())

--- Limpieza de df_vgsales ---
Información del DataFrame df_vgsales después de la limpieza:
<class 'pandas.core.frame.DataFrame'>
Index: 16327 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16327 non-null  int64  
 1   Name          16327 non-null  object 
 2   Platform      16327 non-null  object 
 3   Year          16327 non-null  int64  
 4   Genre         16327 non-null  object 
 5   Publisher     16327 non-null  object 
 6   NA_Sales      16327 non-null  float64
 7   EU_Sales      16327 non-null  float64
 8   JP_Sales      16327 non-null  float64
 9   Other_Sales   16327 non-null  float64
 10  Global_Sales  16327 non-null  float64
dtypes: float64(5), int64(2), object(4)
memory usage: 1.5+ MB

Primeras filas del DataFrame limpio:
   Rank                      Name Platform  Year         Genre Publisher  \
0     1                Wii Sports      Wii  2006        Sports  Ninte

In [23]:
df_vgsales_clean

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


In [26]:
# 2.2. Limpieza de df_movies
print("\n--- Limpieza de df_movies ---")
df_movies_clean = df_movies.copy()

# 1. Se renombra las columnas con caracteres especiales
df_movies_clean = df_movies_clean.rename(columns={'gross(in $)': 'gross_in_dollars'})

# 2. Valores nulos y tipos de datos
# Eliminar filas donde el nombre de la película es nulo
df_movies_clean.dropna(subset=['movie_name'], inplace=True)

# Se limpiea y se convierte 'Year'
# pd.to_numeric convierte a número, y los errores (texto) se convierten en NaT (Not a Time)
df_movies_clean['year'] = pd.to_numeric(df_movies_clean['year'], errors='coerce')
df_movies_clean.dropna(subset=['year'], inplace=True) # Eliminar filas con años no válidos
df_movies_clean['year'] = df_movies_clean['year'].astype(int)

# Se limpia y se convierte 'runtime' (ej. '131 min.' -> 131)
df_movies_clean['runtime'] = df_movies_clean['runtime'].str.extract(r'(\d+)').astype(float)
# Rellenar los nulos de 'runtime' con la media de la columna
runtime_mean = df_movies_clean['runtime'].mean()
df_movies_clean['runtime'] = df_movies_clean['runtime'].fillna(runtime_mean)
df_movies_clean['runtime'] = df_movies_clean['runtime'].astype(int)

# Se rellena 'rating' y 'votes' con la media
rating_mean = df_movies_clean['rating'].mean()
votes_mean = df_movies_clean['votes'].mean()
df_movies_clean['rating'] = df_movies_clean['rating'].fillna(rating_mean)
df_movies_clean['votes'] = df_movies_clean['votes'].fillna(votes_mean)
df_movies_clean['votes'] = df_movies_clean['votes'].astype(int)


# Se rellenan las columnas de texto faltantes
text_cols = ['certificate', 'director', 'star']
for col in text_cols:
    df_movies_clean[col] = df_movies_clean[col].fillna('Unknown')

# La columna 'gross_in_dollars' tiene demasiados nulos para rellenar, la dejamos así por ahora.
# Simplemente rellenamos con 0 para mantener un tipo de dato consistente.
df_movies_clean['gross_in_dollars'] = df_movies_clean['gross_in_dollars'].fillna(0)


# 3. Verificación
print("Información del DataFrame df_movies después de la limpieza:")
df_movies_clean.info()

print("\nPrimeras filas del DataFrame limpio:")
print(df_movies_clean.head())


--- Limpieza de df_movies ---
Información del DataFrame df_movies después de la limpieza:
<class 'pandas.core.frame.DataFrame'>
Index: 302236 entries, 0 to 368299
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   movie_id          302236 non-null  object 
 1   movie_name        302236 non-null  object 
 2   year              302236 non-null  int64  
 3   certificate       302236 non-null  object 
 4   runtime           302236 non-null  int64  
 5   genre             302236 non-null  object 
 6   rating            302236 non-null  float64
 7   description       302236 non-null  object 
 8   director          302236 non-null  object 
 9   director_id       299440 non-null  object 
 10  star              302236 non-null  object 
 11  star_id           293627 non-null  object 
 12  votes             302236 non-null  int64  
 13  gross_in_dollars  302236 non-null  float64
dtypes: float64(2), int64(3), objec

In [27]:
# 2.3. Limpieza de df_yelp_reviews
print("\n--- Limpieza de df_yelp_reviews ---")
df_yelp_clean = df_yelp_reviews.copy()

# 1. Se seleccionan de columnas
# Nos quedamos con las columnas más relevantes para nuestro análisis
df_yelp_clean = df_yelp_clean[['review_id', 'business_id', 'stars', 'text', 'date']]

# 2. Se limpia el texto
# Se conveirte todo el texto de las reseñas a minúsculas
df_yelp_clean['text'] = df_yelp_clean['text'].str.lower()

# 3. Verificación
print("Información del DataFrame df_yelp_reviews después de la limpieza:")
df_yelp_clean.info()
print("\nPrimeras filas del DataFrame limpio:")
print(df_yelp_clean.head())


--- Limpieza de df_yelp_reviews ---
Información del DataFrame df_yelp_reviews después de la limpieza:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   review_id    500000 non-null  object        
 1   business_id  500000 non-null  object        
 2   stars        500000 non-null  int64         
 3   text         500000 non-null  object        
 4   date         500000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 19.1+ MB

Primeras filas del DataFrame limpio:
                review_id             business_id  stars  \
0  KU_O5udG6zpxOg-VcAEodg  XQfwVwDr-v0ZS3_CbbE5Xw      3   
1  BiTunyQ73aT9WBnpR9DZGw  7ATYjTIgM3jUlt4UM3IypQ      5   
2  saUsX_uimxRlCVr67Z4Jig  YjUWPpI6HXG530lwP-fb2A      3   
3  AqPFMleE6RsU23_auESxiA  kxX2SOes4o-D3ZQBkiMRfA      5   
4  Sx8TMOWLNuJBWer-0pcmoA  e4

In [29]:
# 2.4. Limpieza de df_metacritic
print("\n--- Limpieza de df_metacritic ---")
df_metacritic_clean = df_metacritic.copy()

# 1. Se realiza la corrección de los tipos de datos
# Convertir 'metascore' a numérico. 'tbd' (to be determined) se convertirá en NaN.
df_metacritic_clean['metascore'] = pd.to_numeric(df_metacritic_clean['metascore'], errors='coerce')

# Se convierte 'release_date' a formato de fecha
df_metacritic_clean['release_date'] = pd.to_datetime(df_metacritic_clean['release_date'], errors='coerce')

# 2. Se corrige nulos (los que se crearon al forzar la conversión)
# Se rellenan los metascores nulos con 0 o la media, optamos por la media.
metascore_mean = df_metacritic_clean['metascore'].mean()
df_metacritic_clean['metascore'] = df_metacritic_clean['metascore'].fillna(metascore_mean)
df_metacritic_clean['metascore'] = df_metacritic_clean['metascore'].astype(int)

# 3. Verificación
print("Información del DataFrame df_metacritic después de la limpieza:")
df_metacritic_clean.info()
print("\nPrimeras filas del DataFrame limpio:")
print(df_metacritic_clean.head())


--- Limpieza de df_metacritic ---
Información del DataFrame df_metacritic después de la limpieza:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2376 entries, 0 to 2375
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   title         2376 non-null   object        
 1   metascore     2376 non-null   int64         
 2   release_date  2376 non-null   datetime64[ns]
 3   summary       2376 non-null   object        
 4   url           2376 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 92.9+ KB

Primeras filas del DataFrame limpio:
                          title  metascore release_date  \
0  Disco Elysium: The Final Cut         97   2021-03-30   
1                   Half-Life 2         96   2004-11-16   
2            Grand Theft Auto V         96   2015-04-13   
3               Baldur's Gate 3         96   2023-08-03   
4                The Orange Box         96

In [31]:
# Parte 3: Realizar Análisis de Sentimientos
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd

# 1. Se inicializa el analizador de sentimientos
analyzer = SentimentIntensityAnalyzer()

# 2. Función para clasificar el sentimiento
def get_sentiment(text):
    # El 'compound score' es una métrica de -1 (muy negativo) a +1 (muy positivo)
    score = analyzer.polarity_scores(text)['compound']
    if score >= 0.05:
        return 'Positivo'
    elif score <= -0.05:
        return 'Negativo'
    else:
        return 'Neutral'

# Análisis de Reseñas de Yelp
print("--- Iniciando análisis de sentimientos en reseñas de Yelp... ---")
# Demora por la cantidad de datos (500.000)
df_yelp_clean['sentiment'] = df_yelp_clean['text'].apply(get_sentiment)
print("Análisis de Yelp completado.")

# Análisis de Resúmenes de Metacritic (descripción)
print("\n--- Iniciando análisis de sentimientos en resúmenes de Metacritic... ---")
df_metacritic_clean['sentiment'] = df_metacritic_clean['summary'].apply(get_sentiment)
print("Análisis de Metacritic completado.")


# Análisis Rápido de los Resultados
print("\n--- Resultados del Análisis de Sentimientos ---")

# Conteo de tipos de sentimiento en las reseñas de Yelp
print("\nDistribución de sentimientos en Yelp:")
print(df_yelp_clean['sentiment'].value_counts())

# Relación entre el sentimiento de la reseña y las estrellas dadas
print("\nPromedio de estrellas por tipo de sentimiento en Yelp:")
print(df_yelp_clean.groupby('sentiment')['stars'].mean())

# Conteo de tipos de sentimiento en los resúmenes de Metacritic
print("\nDistribución de sentimientos en Metacritic:")
print(df_metacritic_clean['sentiment'].value_counts())

--- Iniciando análisis de sentimientos en reseñas de Yelp... ---
Análisis de Yelp completado.

--- Iniciando análisis de sentimientos en resúmenes de Metacritic... ---
Análisis de Metacritic completado.

--- Resultados del Análisis de Sentimientos ---

Distribución de sentimientos en Yelp:
sentiment
Positivo    430401
Negativo     63577
Neutral       6022
Name: count, dtype: int64

Promedio de estrellas por tipo de sentimiento en Yelp:
sentiment
Negativo    1.696903
Neutral     2.550648
Positivo    4.135848
Name: stars, dtype: float64

Distribución de sentimientos en Metacritic:
sentiment
Positivo    1270
Negativo     979
Neutral      127
Name: count, dtype: int64


In [32]:
df_yelp_clean

,review_id,business_id,stars,text,date,sentiment
0,KU_O5udG6zpxOg-VcAEodg,XQfwVwDr-v0ZS3_CbbE5Xw,3,"if you decide to eat here, just be aware it is...",2018-07-07 22:09:11,Positivo
1,BiTunyQ73aT9WBnpR9DZGw,7ATYjTIgM3jUlt4UM3IypQ,5,i've taken a lot of spin classes over the year...,2012-01-03 15:28:18,Positivo
2,saUsX_uimxRlCVr67Z4Jig,YjUWPpI6HXG530lwP-fb2A,3,family diner. had the buffet. eclectic assortm...,2014-02-05 20:30:30,Positivo
3,AqPFMleE6RsU23_auESxiA,kxX2SOes4o-D3ZQBkiMRfA,5,"wow! yummy, different, delicious. our favo...",2015-01-04 00:01:03,Positivo
4,Sx8TMOWLNuJBWer-0pcmoA,e4Vwtrqf-wpJfwesgvdgxQ,4,cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,Positivo
...,...,...,...,...,...,...
499995,q4Z6NCR1IMRdpPorTD89Tg,-QwHN9KoluPcA0YFllwFYQ,5,we won the playoff game last night and is sche...,2021-06-30 16:14:03,Positivo
499996,KTg0i04SbVWGWHHJmGOI_A,t_v2TyjeqaRkrfZKudY9cA,5,we have been a resident almost 4 years at manz...,2021-03-24 20:21:27,Positivo
499997,ZuMNAPcArFtaGufe-nwGOA,3XirYkP9PJvVXIEDPNNXLA,4,this place is hyped as one of the best places ...,2021-07-03 04:51:07,Positivo
499998,Lohri9uZyvoNnH5z_NT6DA,gfPDLZimZu1NtBIDbeXetg,2,saw the reviews so thought i'd try this place....,2019-04-16 23:28:41,Positivo


In [36]:
# Parte 5: Conectar y cargar datos a SQL Server

# Credenciales de conexión
server = 'proyecto-analisis-de-datos.database.windows.net'  # Ej: mi-servidor-epn.database.windows.net
database = 'ProyectoFinalDB'          # Ej: ProyectoAnalitica
username = 'admin-epn'                 # Ej: admin_user
password = 'Brandon.2.0'
driver = 'ODBC Driver 17 for SQL Server' # No cambiar esto

# Contrucción del motor de conexión con SQLAlchemy
# Este formato de URL es el que entiende SQLAlchemy para conectarse
connection_string = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver}"
engine = sqlalchemy.create_engine(connection_string)

print("Motor de SQLAlchemy creado. Intentando conectar...")

try:
    # Se carga cada DataFrame a una tabla en SQL Server
    # Se usa un bloque try-except para manejar posibles errores de conexión
    with engine.connect() as conn:
        print("¡Conexión a SQL Server exitosa!")

        # 1. Se carga df_vgsales_clean
        print("Cargando 'df_vgsales_clean' a la tabla 'VentasVideojuegos'...")
        df_vgsales_clean.to_sql('VentasVideojuegos', con=conn, if_exists='replace', index=False, chunksize=1000)
        print("Tabla 'VentasVideojuegos' cargada.")

        # 2. Se carga df_movies_clean
        print("Cargando 'df_movies_clean' a la tabla 'Peliculas'...")
        # Usamos chunksize para manejar la memoria con DataFrames grandes
        df_movies_clean.to_sql('Peliculas', con=conn, if_exists='replace', index=False, chunksize=1000)
        print("Tabla 'Peliculas' cargada.")

        # 3. Se carga df_yelp_clean
        print("Cargando 'df_yelp_clean' a la tabla 'ResenasYelp'...")
        df_yelp_clean.to_sql('ResenasYelp', con=conn, if_exists='replace', index=False, chunksize=1000)
        print("Tabla 'ResenasYelp' cargada.")

        # 4. Se carga df_metacritic_clean
        print("Cargando 'df_metacritic_clean' a la tabla 'ResenasMetacritic'...")
        df_metacritic_clean.to_sql('ResenasMetacritic', con=conn, if_exists='replace', index=False, chunksize=1000)
        print("Tabla 'ResenasMetacritic' cargada.")

        print("\n¡Proceso de carga a SQL Server completado!")

        # Verificación
        print("\nVerificando datos: Leyendo las 5 primeras filas de la tabla 'VentasVideojuegos' desde SQL Server...")
        df_from_sql = pd.read_sql("SELECT TOP 5 * FROM VentasVideojuegos", conn)
        print(df_from_sql)

except Exception as e:
    print(f"\nError durante la conexión o carga a SQL Server: {e}")
    print("Por favor, verifica tus credenciales, la configuración del firewall y que el servidor esté activo.")

Motor de SQLAlchemy creado. Intentando conectar...
¡Conexión a SQL Server exitosa!
Cargando 'df_vgsales_clean' a la tabla 'VentasVideojuegos'...
Tabla 'VentasVideojuegos' cargada.
Cargando 'df_movies_clean' a la tabla 'Peliculas'...
Tabla 'Peliculas' cargada.
Cargando 'df_yelp_clean' a la tabla 'ResenasYelp'...
Tabla 'ResenasYelp' cargada.
Cargando 'df_metacritic_clean' a la tabla 'ResenasMetacritic'...
Tabla 'ResenasMetacritic' cargada.

¡Proceso de carga a SQL Server completado!

Verificando datos: Leyendo las 5 primeras filas de la tabla 'VentasVideojuegos' desde SQL Server...
   Rank                      Name Platform  Year         Genre Publisher  \
0     1                Wii Sports      Wii  2006        Sports  Nintendo   
1     2         Super Mario Bros.      NES  1985      Platform  Nintendo   
2     3            Mario Kart Wii      Wii  2008        Racing  Nintendo   
3     4         Wii Sports Resort      Wii  2009        Sports  Nintendo   
4     5  Pokemon Red/Pokemon Blue

In [37]:
# Parte 6: Relacionar la información de cada negocio registrado en Yelp y volverlo a cargar a SQL Server

print("--- Cargando el dataset de negocios de Yelp ---")
try:
    # Se carga el archivo JSON de negocios
    df_business = pd.read_json('yelp_academic_dataset_business.json', lines=True)

    # Se selecciona solo las columnas que necesitamos
    df_business_selection = df_business[['business_id', 'name', 'city', 'state', 'categories']]

    print("Dataset de negocios cargado y filtrado exitosamente.")
    print(df_business_selection.head())

except FileNotFoundError:
    print("Error: Asegúrate de haber subido el archivo 'yelp_academic_dataset_business.json' a Colab.")
except Exception as e:
    print(f"Ocurrió un error: {e}")

--- Cargando el dataset de negocios de Yelp ---
Dataset de negocios cargado y filtrado exitosamente.
              business_id                      name           city state  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ  Santa Barbara    CA   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store         Affton    MO   
2  tUFrWirKiKi_TAnsVWINQQ                    Target         Tucson    AZ   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   Philadelphia    PA   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery     Green Lane    PA   

                                          categories  
0  Doctors, Traditional Chinese Medicine, Naturop...  
1  Shipping Centers, Local Services, Notaries, Ma...  
2  Department Stores, Shopping, Fashion, Home & G...  
3  Restaurants, Food, Bubble Tea, Coffee & Tea, B...  
4                          Brewpubs, Breweries, Food  


In [38]:
# Realiza la unión
print("\n--- Uniendo los DataFrames de reseñas y negocios ---")

# Hacemos el 'merge' (unión) usando 'business_id' como la clave
# 'how='left'' asegura que se mantenga todas las reseñas de la tabla izquierda (df_yelp_clean)
df_yelp_final = pd.merge(df_yelp_clean, df_business_selection, on='business_id', how='left')

print("¡Unión completada!")


--- Uniendo los DataFrames de reseñas y negocios ---
¡Unión completada!


In [40]:
# Verficación
print("\n--- Verificación del DataFrame final de Yelp ---")

# Muestra la información para ver las nuevas columnas
df_yelp_final.info()

# Muestra las primeras filas para ver cómo se ven los datos combinados
# Ahora deberías ver el nombre del negocio y su ciudad junto a cada reseña
print("\nPrimeras 5 filas del DataFrame combinado:")
df_yelp_final


--- Verificación del DataFrame final de Yelp ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   review_id    500000 non-null  object        
 1   business_id  500000 non-null  object        
 2   stars        500000 non-null  int64         
 3   text         500000 non-null  object        
 4   date         500000 non-null  datetime64[ns]
 5   sentiment    500000 non-null  object        
 6   name         500000 non-null  object        
 7   city         500000 non-null  object        
 8   state        500000 non-null  object        
 9   categories   499972 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(8)
memory usage: 38.1+ MB

Primeras 5 filas del DataFrame combinado:


,review_id,business_id,stars,text,date,sentiment,name,city,state,categories
0,KU_O5udG6zpxOg-VcAEodg,XQfwVwDr-v0ZS3_CbbE5Xw,3,"if you decide to eat here, just be aware it is...",2018-07-07 22:09:11,Positivo,Turning Point of North Wales,North Wales,PA,"Restaurants, Breakfast & Brunch, Food, Juice B..."
1,BiTunyQ73aT9WBnpR9DZGw,7ATYjTIgM3jUlt4UM3IypQ,5,i've taken a lot of spin classes over the year...,2012-01-03 15:28:18,Positivo,Body Cycle Spinning Studio,Philadelphia,PA,"Active Life, Cycling Classes, Trainers, Gyms, ..."
2,saUsX_uimxRlCVr67Z4Jig,YjUWPpI6HXG530lwP-fb2A,3,family diner. had the buffet. eclectic assortm...,2014-02-05 20:30:30,Positivo,Kettle Restaurant,Tucson,AZ,"Restaurants, Breakfast & Brunch"
3,AqPFMleE6RsU23_auESxiA,kxX2SOes4o-D3ZQBkiMRfA,5,"wow! yummy, different, delicious. our favo...",2015-01-04 00:01:03,Positivo,Zaika,Philadelphia,PA,"Halal, Pakistani, Restaurants, Indian"
4,Sx8TMOWLNuJBWer-0pcmoA,e4Vwtrqf-wpJfwesgvdgxQ,4,cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,Positivo,Melt,New Orleans,LA,"Sandwiches, Beer, Wine & Spirits, Bars, Food, ..."
...,...,...,...,...,...,...,...,...,...,...
499995,q4Z6NCR1IMRdpPorTD89Tg,-QwHN9KoluPcA0YFllwFYQ,5,we won the playoff game last night and is sche...,2021-06-30 16:14:03,Positivo,Spike's Trophies,Philadelphia,PA,"Printing Services, Shopping, Screen Printing/T..."
499996,KTg0i04SbVWGWHHJmGOI_A,t_v2TyjeqaRkrfZKudY9cA,5,we have been a resident almost 4 years at manz...,2021-03-24 20:21:27,Positivo,Manzanita Gate Apartments,Reno,NV,"Apartments, Home Services, Real Estate"
499997,ZuMNAPcArFtaGufe-nwGOA,3XirYkP9PJvVXIEDPNNXLA,4,this place is hyped as one of the best places ...,2021-07-03 04:51:07,Positivo,Crown Candy Kitchen,St. Louis,MO,"Specialty Food, Food, American (Traditional), ..."
499998,Lohri9uZyvoNnH5z_NT6DA,gfPDLZimZu1NtBIDbeXetg,2,saw the reviews so thought i'd try this place....,2019-04-16 23:28:41,Positivo,Nirvana Indian Bistro,Lafayette Hill,PA,"Restaurants, Indian"


In [46]:
# Reescribe la tabla de Yelp en SQL Server

print("Cargando el DataFrame final de Yelp a la tabla 'ResenasYelp'...")

# Usamos if_exists='replace' para borrar la tabla vieja y crear la nueva
df_yelp_final.to_sql('ResenasYelp', 
                     con=engine.connect(), 
                     if_exists='replace',
                     index=False, 
                     chunksize=1000)

print("¡Tabla 'ResenasYelp' reescrita exitosamente con la información de los negocios!")

Cargando el DataFrame final de Yelp a la tabla 'ResenasYelp'...
¡Tabla 'ResenasYelp' reescrita exitosamente con la información de los negocios!
